GAMA Hosts paper

Author: Maria B. Stone

# Get a table with sSFR data for normal galaxies

**This code will give a table with sSFR data from CIGALE and MAGPHYS for normal galaxies.**

-------

Here we import the necessary packages.

In [1]:
import numpy as np
from astropy import units as u
from astropy.table import Table, join

from functions_GAMA import controlgxs_join_magphys_cigale_results

# for plotting
from matplotlib import pyplot as plt
%matplotlib inline


Data tables are stored in the directory '1_tables'.
They will be called from inside it in the code.

In [2]:
DATA_DIR = "../1_tables/"

Join the data from CIGALE and MAGPHYS for each normal galaxy in the control group, based on 'CATAID'.

In [3]:
# Specify the file containing the data from CIGALE for normal galaxies
cigale_file = DATA_DIR + 'results_galaxies.fits'
#cigale_file = Table.read(DATA_DIR + 'table_data_controlgxs_cigale_subsample.fits')

In [4]:
# Specify the file with MAGPHYS data for a single set of normal galaxies, which is our control
magphys_file = DATA_DIR + '12_data_seed_gals_set199_magphys.fits'
#magphys_file = Table.read(DATA_DIR + 'table_data_controlgxs_magphys_subsample.fits')

In [5]:
# Join the tables
data = controlgxs_join_magphys_cigale_results(cigale_file, magphys_file, id_col='id')

----------

Now that we have everything organized, let's pull the sSFR data that we need.

First get the data for MAGPHYS estimate for sSFR for a normal galaxy

In [6]:
sSFR_magphys = data['sSFR_0_1Gyr_best_fit']

For CIGALE, we need to calculate ourselves, 

``` mathematica 
specefic SFR = SFR / stellar Mass 
```

In [7]:
#SFR estimate from CIGALE
sfr_cigale = data['bayes.sfh.sfr']

#Mass estimate from CIGALE
mass_cigale = data['bayes.stellar.m_star']

# Calculate the sSFR based on the formula above
sSFR_cigale_value = sfr_cigale.value / mass_cigale.value # The units are 1/yr

Calculate the log10 values

In [8]:
sSFR_cigale_log_value = np.log10(sSFR_cigale_value) # The units are 1/yr
sSFR_magphys_log = np.log10(sSFR_magphys)

We create a table for working with sSFR of normal galaxies and save it.

In [9]:
sSFR_table = Table()

sSFR_table['CATAID'] = np.copy(data['CATAID'])
sSFR_table['log_sSFR_cigale'] = sSFR_cigale_log_value * (1/u.yr) # I specify the unit
sSFR_table['log_sSFR_magphys'] = sSFR_magphys_log

In [10]:
sSFR_table.write('table_data_controlgxs_sSFR.fits', overwrite=True)

In [11]:
# End